# Source Code Interventions

## Summary

You can use NNsight to access the intermediate vales of a forward pass! We can print the `.source` of a module to see its forward pass and associated names of each operation.

```python
# .source to print the first transformer
print(model.transformer.h[0].attn.source)
```

You can also use `source` to get and set intermediate values during the tracing context.

```python
with model.trace(prompt):
  # get intermediate value
  attention = model.transformer.h[0].attn.source.attention_interface_0.output.save()

  # set intermediate value
  model.transformer.h[k].attn.source.attention_interface_0.output = attention
```

## When to Use

One of the most common use-cases for `source` is accessing attention.

## How to Use

To view the intermediate values of a forward pass, you can call `.source` on a module. Printing `.source` of a module will help you see the names of the forward pass operations.

In [5]:
from nnsight import LanguageModel

model = LanguageModel('openai-community/gpt2', device_map='auto')

print(model.transformer.h[0].attn.source)

                                       * def forward(
                                       0     self,
                                       1     hidden_states: Optional[tuple[torch.FloatTensor]],
                                       2     past_key_value: Optional[Cache] = None,
                                       3     cache_position: Optional[torch.LongTensor] = None,
                                       4     attention_mask: Optional[torch.FloatTensor] = None,
                                       5     head_mask: Optional[torch.FloatTensor] = None,
                                       6     encoder_hidden_states: Optional[torch.Tensor] = None,
                                       7     encoder_attention_mask: Optional[torch.FloatTensor] = None,
                                       8     output_attentions: Optional[bool] = False,
                                       9     **kwargs,
                                      10 ) -> tuple[Union[torch.Tensor, tuple[torc

Okay, so you can now see the operations within the attention. But, how would you actually access and intervene on these operations?

Within the tracing context, you can access and intervene on `source` values as you do with getting and setting operations on modules.

Let's try to access the output of `attention_interface_0` and save it. 

In [6]:
prompt = "The Eiffel Tower is located in the city of"

with model.trace(prompt):

  attention = model.transformer.h[0].attn.source.attention_interface_0.output.save()

print(attention)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

(tensor([[[[-1.3066e-02, -1.4464e-02,  1.2694e-01,  ..., -4.9182e-02,
            1.0464e-01,  2.3067e-02],
          [ 5.9014e-01,  1.0051e-01, -2.0716e-01,  ..., -6.9383e-01,
           -2.7763e-01,  2.0517e-01],
          [-2.8404e-02, -1.1449e-01, -2.1676e-02,  ...,  3.9217e-03,
            7.8844e-02, -3.9936e-03],
          ...,
          [-1.0771e-01, -2.1316e-01, -2.1841e-02,  ..., -2.3210e-01,
            2.1270e-02, -6.6547e-02],
          [ 9.3223e-02, -1.0404e-01, -2.1104e-01,  ...,  1.8502e-01,
            2.2378e-01, -3.1989e-02],
          [-2.4770e-02, -3.7828e-01,  1.1838e-01,  ...,  1.1582e-02,
           -2.4843e-01, -1.1559e-01]],

         [[ 1.6556e-02, -1.4358e-02,  1.2063e-01,  ..., -4.5132e-02,
            9.7916e-02,  2.9936e-02],
          [ 6.3335e-01,  1.1630e-01,  2.4260e-01,  ...,  1.8957e-01,
            8.8069e-02, -5.1060e-02],
          [-3.0525e-02, -8.4757e-02, -3.9247e-02,  ..., -9.0845e-04,
            6.1980e-02, -5.6949e-03],
          ...,
    

We can also set values to intervene on intermediate variables.

In [11]:
with model.trace(prompt):
  attention_0 = model.transformer.h[0].attn.source.attention_interface_0.output.save()

  # change attention of layer 7 to that of layer 0
  model.transformer.h[7].attn.source.attention_interface_0.output = attention_0

Inside of the tracing context, `source` also works recursively:

In [7]:
with model.trace(prompt):

  print(model.transformer.h[0].attn.source.attention_interface_0.source)

                                                        * def sdpa_attention_forward(
                                                        0     module: torch.nn.Module,
                                                        1     query: torch.Tensor,
                                                        2     key: torch.Tensor,
                                                        3     value: torch.Tensor,
                                                        4     attention_mask: Optional[torch.Tensor],
                                                        5     dropout: float = 0.0,
                                                        6     scaling: Optional[float] = None,
                                                        7     is_causal: Optional[bool] = None,
                                                        8     **kwargs,
                                                        9 ) -> tuple[torch.Tensor, None]:
 kwargs_get_0                            

## Related

*   Getting
*   Setting

